https://wikidocs.net/80437
https://blog.naver.com/gkswns3708/222811466198

In [1]:
import torch.nn as nn
from einops import rearrange
from torch import Tensor
import torch
from torch.utils.data import DataLoader
from dataset import ImageLayout
from extractor.image import ImageEncoder
from extractor.positional_encoding import build_position_encoding_1d

In [2]:
data = ImageLayout(type='train')
loader = DataLoader(data, batch_size=2)
sample = next(iter(loader))

In [3]:
from cgllike import CLDM

In [4]:
model = CLDM()

/conda/joonsm/envs/CLDM/lib/python3.9/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/conda/joonsm/envs/CLDM/lib/python3.9/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/conda/joonsm/envs/CLDM/lib/python3.9/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/conda/joonsm/envs/CLDM/lib/python3.9/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
/conda/joonsm/envs/CLDM/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_

In [5]:
c = model(sample, 1)

In [7]:
from diffusers import DDPMScheduler

In [8]:
noise_scheduler = DDPMScheduler(num_train_timesteps=250, prediction_type='sample', clip_sample=True)

In [9]:
bsz = sample['box'].shape[0]

In [10]:
t= torch.randint(0,250,(bsz,))

In [11]:
noise = torch.randn(sample['box'].shape)
b = noise_scheduler.add_noise(sample['box'],noise, t)

In [12]:
sample['box'].size()

torch.Size([2, 4])

In [13]:
b.size()

torch.Size([2, 4])

In [14]:
decode = nn.Linear(256,1)

In [15]:
b= torch.randn(256,4,256)

In [16]:
ddd = torch.randn(256,4,1)

In [17]:
ddd.squeeze(-1).size()

torch.Size([256, 4])